# 用于新闻问答的 agent - v0.1

存在的问题：

- 不存在文档的情况下，llm自由发挥回答 -- 应该告知未找到相关信息
- 嵌入使用在哪里了？为啥没有score
- 使用llm解析json格式的问题，存在解析问题

初步结论：

- 缺陷很多，比如不根据文档数据回答

## 加载模型和嵌入

In [1]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 3.28 s, sys: 452 ms, total: 3.73 s
Wall time: 3.36 s


## 根据url生成新闻数据的函数

In [2]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    result['url']=url
    return result
    

CPU times: user 8.37 ms, sys: 4.06 ms, total: 12.4 ms
Wall time: 11.8 ms


In [3]:
get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")

{'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”',
 'author': '齐倩',
 'publish_time': '2024-06-20 15:24:32',
 'content': '【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批朝觐者死亡。由于官方没有公布具体数据，死亡人数尚不确定。法新社最新统计称，截至当地时间6月19日，死亡人数已飙升至900多人。\n朝觐是伊斯兰教的五大基本功课之一，所有身体健全且有经济能力的穆斯林在一生之中都须完成至少一次朝觐。据伊斯兰历法，朝觐季节每年都会变化，今年是在六月，也是沙特最热的月份之一。\n沙特官方数据显示，来自世界各地的约180万穆斯林参加了为期数天的朝觐活动，其中约160万人来自国外，不乏年老体弱的人。据介绍，每年的朝觐活动都会吸引数十万来自低收入国家的朝觐者，“其中许多人几乎没有接受过朝觐前的医疗保健”，导致传染病传播。\n另外，每年都有数以万计的朝觐者为了省钱，试图在没有获得官方签证的情况下参加朝觐。沙特近期的酷热高温，给朝觐活动又增添危险：非法朝觐者无法使用朝觐路线上提供的空调设施等降暑服务。\n当地时间17日，麦加气温达到51.8摄氏度。与此同时，沙特以及其他国家的民众纷纷前往医院，或者在社交媒体焦急地发帖求助，试图寻找朝觐期间失踪的亲友。\n麦加气温超过50摄氏度，大批朝觐者死于高温（图自视觉中国，下同）\n沙特没有提供死亡人数的具体信息，但仅在上周日（16日）就报告了2700多例“中暑”病例。由于缺乏官方数据，死亡人数尚未确定，且可能会继续上升。\n美联社19日报道称，网上流传的一份名单显示，数日内已有至少550人死亡。法新社则对各国公布数据统计得出，截至当地时间19日，死亡人数已飙升至900多人。一名阿拉伯地区的外交官告诉法新社，仅埃及人的死亡人数就从18日的300多人跃升至19日的“至少600人”，其中大部分死于高温。\n还有一名外交官透露，除了一人在小规模人群挤压中受重伤致死外，“所有埃及人均因高温死亡”。他还表示，死亡总数来自麦加当地医院太平间的数据。\n当地时间18日，埃及外交部曾发布声明表示，埃及正在与沙特官方合作，搜寻在朝觐期间失踪的埃及人。除埃及外，约旦、印度尼西亚、印度、伊朗、塞内加尔、突尼斯和伊拉

## 初始化文档摘要索引

In [4]:
%%time

!rm -rf ./index

CPU times: user 0 ns, sys: 11.6 ms, total: 11.6 ms
Wall time: 120 ms


In [5]:
%%time

import os
from llama_index.core import(
    StorageContext, 
    load_index_from_storage,
    get_response_synthesizer,
    DocumentSummaryIndex,
    Document
)
from llama_index.core.node_parser import SentenceSplitter

def _load_index(folder_path):
    # 考虑不存在的情况，创建空的索引
    if not os.path.exists(folder_path):
        splitter = SentenceSplitter(chunk_size=1024)
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize", use_async=True,
        )
        
        doc_summary_index = DocumentSummaryIndex.from_documents(
            [],
            transformations=[splitter],
            response_synthesizer=response_synthesizer,
            show_progress=True,
        )
        
        doc_summary_index.storage_context.persist(folder_path)
        
    storage_context = StorageContext.from_defaults(persist_dir=folder_path)
    return load_index_from_storage(storage_context)
    

CPU times: user 19 µs, sys: 3 µs, total: 22 µs
Wall time: 23.1 µs


In [6]:
%%time

doc_summary_index=_load_index('index')

Parsing nodes: 0it [00:00, ?it/s]

Summarizing documents: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

CPU times: user 284 ms, sys: 28.3 ms, total: 312 ms
Wall time: 309 ms


## 新增一条新闻文档到索引

### 插入数据并保存

In [7]:
%%time

news_data=get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")
news_data['content'][:100]

CPU times: user 103 ms, sys: 3.81 ms, total: 106 ms
Wall time: 212 ms


'【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批朝觐者死亡。由于官方没有公布具体数据，死亡人数尚不确定。法新社最新统计称，截至当地时间6月19日，死亡人数'

In [8]:
%%time

def _data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [9]:
%%time

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

current doc id: 沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”
CPU times: user 77.5 ms, sys: 129 µs, total: 77.7 ms
Wall time: 13.7 s


### 根据新闻回答问题

In [10]:
%%time

doc_summary_index=_load_index('index')

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

response = query_engine.query("麦加参加朝觐活动发生了啥？")
response.print_response_stream()

麦加近期的高温导致参加朝觐活动的大批朝觐者死亡。由于官方没有公布具体数据，确切的死亡人数尚不确定。法新社统计显示，在6月19日之前，已有超过900人因高温而死亡。每年都有大量来自低收入国家的朝觐者参与，许多人可能缺乏接受过朝觐前的医疗保健，这使得传染病传播成为问题。此外，为了省钱，数以万计的朝觐者试图在没有获得官方签证的情况下参加活动。

沙特近期的酷热高温给朝觐活动增添了危险，非法朝觐者无法使用朝觐路线上提供的空调设施等降暑服务。麦加气温超过50摄氏度，导致大批朝觐者死于高温。据报道，在6月17日当天，麦加气温达到51.8摄氏度。

沙特官方报告了2700多例“中暑”病例，并表示死亡人数可能会继续上升。网上流传的一份名单显示，数日内已有至少550人死亡，法新社统计的死亡人数已飙升至900多人。其中，埃及人的死亡人数从18日的300多人跃升至19日的“至少600人”，大部分死于高温。

沙特官方称今年朝觐季取得了“成功”，尽管面临高温挑战和朝觐者众多的情况，并表示先进的冷却系统和持续供应的水源确保了朝觐顺利、安全。然而，许多朝觐仪式在白天举行，朝觐者需要在户外活动数小时，导致有人在路边晕倒，救护车不堪重负。

此外，随着气候变暖，高温致死事件频繁发生，预示着未来几十年数千万朝觐者将面临更大的风险。德国气候分析研究所的科学顾问卡尔-弗里德里希·施洛伊斯纳表示，攀登阿拉法特山是朝觐的重要仪式，在高温天气下已经变得“对人体健康极其危险”。CPU times: user 584 ms, sys: 49 ms, total: 633 ms
Wall time: 11 s


### 生成新闻的摘要

#### 从索引中拿到已经生成的英文摘要

In [12]:
%%time

doc_summary=doc_summary_index.get_document_summary(document.doc_id)
doc_summary

CPU times: user 138 µs, sys: 0 ns, total: 138 µs
Wall time: 141 µs


'The provided text discusses a situation where high temperatures in Saudi Arabia during June have led to deaths among pilgrims participating in Hajj activities in Mecca, particularly those from countries like Egypt and Jordan. The lack of official data on the number of fatalities makes it difficult to determine an exact figure, but reports suggest that over 900 people may have died due to heat-related issues.\n\nThe text highlights several aspects:\n1. **Islamic Pilgrimage**: Hajj is one of the five pillars of Islam and requires all physically capable Muslims who can afford it to perform at least once in their lifetime.\n2. **Temperature Conditions**: The month of June, when Hajj occurs, is known for its extreme heat in Saudi Arabia, which poses significant risks to pilgrims, especially those from low-income countries with limited access to healthcare before the pilgrimage and those without official visas who attempt to enter illegally.\n3. **Health Risks**: Pilgrims face challenges du

#### 基于英文摘要组织中文对话输出

In [13]:
%%time

prompt=f"""
请根据{doc_summary}, 用中文做有条理的表述。
"""
stream_response=Settings.llm.stream_complete(prompt)
for response in stream_response:
    print(response.delta, end="", flush=True)

文本讨论了沙特阿拉伯6月高温导致朝觐者在麦加进行朝圣活动时死亡的情况，尤其是来自埃及和约旦等国的朝觐者。由于官方数据未提供确切的伤亡人数，报道估计可能有超过900人因热相关问题而丧生。

文本中提到了几个关键点：

1. **伊斯兰朝圣**：朝圣是伊斯兰教的五大支柱之一，要求所有身体和经济条件允许的穆斯林至少在一生中完成一次。这表明了朝圣对信仰的重要性以及其对个人和社区的影响。

2. **温度状况**：6月是沙特阿拉伯极端炎热的时候，这对参与朝圣活动的人员构成重大风险，尤其是对于来自低收入国家、在前往朝圣前医疗资源有限的朝觐者，以及那些试图非法入境的没有官方签证的人。

3. **健康风险**：朝觐者面临的主要挑战包括缺乏医疗服务、高温和白天进行户外活动时的极端温度。这些因素增加了他们遭受热相关疾病的风险。

4. **政府反应**：沙特当局报告了数千起中暑病例，但未提供具体死亡人数。他们声称先进的冷却系统和持续的供水确保了朝圣的安全性和顺利进行。

对于上述问题的回答如下：

- 由于缺乏官方数据，确切的伤亡人数无法确定，估计有超过900人可能因高温而丧生。
- 主要健康风险包括中暑、脱水和其他热相关疾病，这些都与极端温度和缺乏适当的医疗支持有关。
- 沙特政府通过报告中暑病例并声称使用先进的冷却系统和持续供水来应对朝圣期间的高温问题。然而，他们并未提供具体的死亡人数数据。
- 为确保安全并预防此类事件，沙特当局采取了措施包括提供冷却设施、确保充足的水资源供应以及可能对朝觐者进行健康检查等。这些措施旨在减少高温带来的风险，并提高朝圣的安全性。

总体而言，文本强调了在极端天气条件下进行朝圣的挑战和风险，以及政府为应对这些问题所做出的努力。CPU times: user 538 ms, sys: 52.4 ms, total: 591 ms
Wall time: 11.4 s


#### 提取摘要中的问题条目

In [19]:
%%time

prompt=f"""
请根据{doc_summary}, 提取文章中的问题，并通过下面示例的json格式输出。

[
    {{'question': 'xxxxxx'}},
]
"""
stream_response=Settings.llm.stream_complete(prompt)
for response in stream_response:
    print(response.delta, end="", flush=True)

[
    {
        'question': '有多少来自不同国家的朝圣者因沙特阿拉伯6月高温在麦加进行朝觐活动时死亡？',
    },
    {
        'question': '朝圣者在沙特阿拉伯朝觐期间面临的主要健康风险是什么？',
    },
    {
        'question': '沙特政府如何应对朝觐期间因高温导致的死亡事件？',
    },
    {
        'question': '沙特当局采取了哪些具体措施来确保安全并防止此类事故再次发生？',
    }
]CPU times: user 137 ms, sys: 20.8 ms, total: 158 ms
Wall time: 5.02 s


### 从索引获取指定文档的信息

In [20]:
doc_summary_index.ref_doc_info

{'沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”': RefDocInfo(node_ids=['0777402f-2fbc-4357-87ae-17f8c0a6ddb7', 'aa764d29-82f8-45cd-8b53-96c5093c09bf', '49c9cfb9-79b8-4a14-b65c-9f9f0fde3e92', '7b7f553f-7067-4bcf-9cb1-0d4a9f184c30', 'e0a4e135-b842-4631-a1eb-f447df41d0b0'], metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']})}

In [21]:
doc_summary_index.ref_doc_info[document.doc_id]

RefDocInfo(node_ids=['0777402f-2fbc-4357-87ae-17f8c0a6ddb7', 'aa764d29-82f8-45cd-8b53-96c5093c09bf', '49c9cfb9-79b8-4a14-b65c-9f9f0fde3e92', '7b7f553f-7067-4bcf-9cb1-0d4a9f184c30', 'e0a4e135-b842-4631-a1eb-f447df41d0b0'], metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']})

## 增加第二条新闻文档到索引

### 加入到索引

In [22]:
%%time

news_data=get_news_data("https://www.guancha.cn/MaoKeJi/2024_06_20_738581.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

doc_summary_index=_load_index('index')

current doc id: 毛克疾：美国究竟能在多大程度上容忍印度？
CPU times: user 166 ms, sys: 3.29 ms, total: 169 ms
Wall time: 44.1 s


### 提取文章蕴含的问题

In [25]:
%%time

doc_summary=doc_summary_index.get_document_summary(document.doc_id)

prompt=f"""
请根据{doc_summary}, 提取文章中的问题，并通过下面示例的json格式输出, 问题文字使用中文。

[
    {{'question': 'xxxxxx'}},
]
"""
stream_response=Settings.llm.stream_complete(prompt)
for response in stream_response:
    print(response.delta, end="", flush=True)

[
    {'question': '1. 为什么美国和印度之间的战略伙伴关系正在增长？',
     'question': '2. 美国对印度的全球策略态度在过去是如何变化的？',
     '3. 中华人民共和国在美印关系中扮演什么角色？',
     '4. 地缘政治冲突如何影响美印双边关系，特别是与中国有关的领土争端？',
     '5. “加勒万山谷冲突”如何深刻地改变了印度精英对中国的态度，并影响了与美国的关系？',
     '6. 在“加勒万山谷冲突”之后，美国和印度在哪些领域扩大了合作？',
     '7. 美国对印度的政策是否因总统的不同而有所不同？',
     '8. 美国如何支持印度的工业发展和技术进步，特别是与中国的比较？',
     '9. 印度对外政策决策中，在与中国和美国的关系之间有何战略考量？',
     '10. 美国如何看待印度的民主价值观，并在外交互动中扮演什么角色？'}
]CPU times: user 269 ms, sys: 39 ms, total: 309 ms
Wall time: 6 s


### 根据新闻回答问题

#### 针对当前新闻的问题

In [44]:
%%time

response = query_engine.query("美国对印度的全球策略态度在过去是如何变化的？")
response.print_response_stream()

美国对印度的全球策略态度在过去经历了从相对谨慎和选择性支持到更深入、全面的战略伙伴关系的变化。在小布什和奥巴马政府时期，尽管中国因素是决定美印关系的基础，但两国的关系更多地处于平行发展的状态，美国并未将中印关系视为主要动力。这一阶段，美国在涉及印度的问题上考虑更多的是如何拉拢印度而非直接针对中国的动机。

转折点出现在2017年的“洞朗对峙”事件后，莫迪政府对美国的态度发生了重大变化，并促使美日印澳四方机制得以复活。随后的“加勒万冲突”成为另一个关键节点，它极大地缓解了印度高层对于美国在中印冲突中的可靠性的疑虑。这一事件导致印度精英层对中国的态度发生深刻变化，并促进了他们在敏感领域与美国的合作。

2020年之后，拜登政府进一步深化了美印合作至产业科技领域，这表明美国对印度的全球策略态度已经从不直接针对中国的拉拢政策转变为共同对抗中国崛起的基础上的战略伙伴关系。在此期间，美印关系在战略层面上紧密相连，并以共同敌意作为接近的基础。

总体来看，在面对“中国因素”时，美印合作的理论上限可能会受到一定影响，但不太可能瞬间改变双方战略合作的实际进度或总体方向。未来，两国将继续围绕这一关键点进行多领域的深度合作与战略互动。CPU times: user 407 ms, sys: 33 ms, total: 440 ms
Wall time: 30 s


In [45]:
len(response.source_nodes)

7

#### 针对之前新闻的问题

In [27]:
%%time

response = query_engine.query("沙特当局采取了哪些具体措施来确保安全并防止此类事故再次发生？")
response.print_response_stream()

沙特当局为了确保朝觐活动的安全，并预防类似事件的再次发生，采取了一系列具体的措施：

1. **先进的冷却系统**：官方提供了高效的降温设施，如在朝圣路线和聚集点安装空调或风扇等设备，以帮助应对高温环境。

2. **持续供应水源**：为朝圣者提供充足的饮用水和其他液体补充，确保他们能够随时获取清洁的水，这是预防中暑的关键措施之一。

3. **志愿者援助**：现场分发冷饮和冰淇淋给朝圣者，这不仅缓解了高温带来的不适，还提供了即时的冷却效果。

4. **加强医疗支持与急救服务**：麦加医疗中心等医疗机构已经做好准备，确保在紧急情况下能够迅速响应并提供必要的医疗救助。

5. **提高公众意识**：通过教育和宣传增强朝圣者对高温环境下的自我保护意识，包括提醒他们避免在最热的时段外出、及时补充水分等。

6. **改善基础设施与规划**：调整了朝圣路线或活动安排，减少人群拥挤和长时间暴露于极端温度下，例如优化仪式时间以避开最热时段。

7. **国际合作与信息共享**：与其他国家合作分享信息和经验教训，特别是在应对高温挑战方面，与医疗、气象或公共卫生领域的专家交流最佳实践。

8. **建立监测和预警系统**：建立了有效的天气监测和预警机制，以便在极端天气条件下提前采取预防措施，并及时通知朝圣者和相关机构做好准备。CPU times: user 397 ms, sys: 78.7 ms, total: 476 ms
Wall time: 26 s


#### 针对索引中不存在的内容提问

In [40]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    # similarity_top_k=1
)

response = query_engine.query("清洁能源给“看不见的50亿人”带来了什么？")
response.print_response_stream()

清洁能源为生活在缺乏传统电力基础设施或受到污染影响地区的"看不见的50亿人"提供了多方面的改善和益处：

1. **健康状况提升**：通过减少对传统燃料（如木柴、煤炭）的依赖，降低了室内空气污染水平，从而减少了呼吸系统疾病、心脏病等健康问题的发生率。

2. **生活质量提高**：稳定的电力供应为基本服务提供支持，包括照明、充电电子设备和运行家电，显著提升了居民的生活质量。

3. **教育机会增加**：清洁能源确保了学校和社区中心能够持续供电，促进了远程学习的可能性，并提高了整体教育水平。

4. **经济机遇增长**：清洁能源项目创造了就业机会，尤其是在发展中国家，有助于减少贫困，促进当地经济发展。

5. **增强能源安全与独立性**：通过利用本地可再生能源资源，社区减少了对化石燃料进口的依赖，提升了能源安全并降低了因国际油价波动带来的经济风险。

6. **支持可持续发展目标**：清洁能源是实现联合国可持续发展目标的关键工具之一，特别是在清洁饮水、卫生设施、优质教育和经济增长等领域。

7. **促进性别平等**：减轻了女性在获取家庭能源方面的负担，为她们提供了更多参与教育、工作或社区活动的时间。

8. **减少碳排放**：作为零排放或低排放的能源来源，清洁能源有助于减缓全球气候变化的影响，并对环境产生积极影响。CPU times: user 386 ms, sys: 43.7 ms, total: 430 ms
Wall time: 29.1 s


In [41]:
len(response.source_nodes)

7

In [42]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=1
)

response = query_engine.query("清洁能源给“看不见的50亿人”带来了什么？")
response.print_response_stream()

清洁能源为全球未被传统电网覆盖或缺乏可靠电力供应地区的居民，即通常所说的“看不见的50亿人”，提供了清洁、可负担和可持续的能源解决方案。这些群体主要包括生活在偏远地区、农村或经济欠发达地区的居民。清洁能源技术如太阳能、风能、小水电等对这一人群带来了以下几方面的积极影响：

1. **改善生活质量**：稳定的电力供应使得人们能够使用照明、充电设备（如手机和其他小型电器），进行烹饪和日常活动，从而提高生活便利性和安全性。

2. **健康与福祉提升**：减少对传统燃料的依赖，尤其是木柴等固体燃料，显著降低了因室内空气污染导致的呼吸系统疾病和其他健康问题。

3. **教育机会增加**：确保儿童能够在家或学校使用电子设备进行学习，特别是在偏远地区，这有助于提高教育水平和未来就业机会。

4. **经济机遇促进**：为当地社区提供了新的商业机会，如太阳能安装、维护和运营服务，以及小型企业（如移动充电站）的建立，从而促进了经济发展。

5. **减少碳排放**：清洁能源替代化石燃料使用，有助于减缓全球气候变化，对环境产生积极影响。

通过投资清洁能源项目，国际组织、政府和私营部门正在共同努力，确保这些人群能够获得清洁、可靠且负担得起的能源服务。这不仅改善了他们的生活质量，还促进了可持续发展和全球社会的公平性。CPU times: user 416 ms, sys: 43.3 ms, total: 459 ms
Wall time: 27.5 s


In [43]:
len(response.source_nodes)

7

In [37]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='a45e8f6e-db4b-4b86-a103-528c79436834', embedding=None, metadata={'title': '毛克疾：美国究竟能在多大程度上容忍印度？', 'publish_time': '2024-06-20 08:26:59', 'author': '郭涵', 'url': 'https://www.guancha.cn/MaoKeJi/2024_06_20_738581.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/19/20240619133335691.jpg', 'https://i.guancha.cn/news/external/2024/06/19/20240619133601871.jpg', 'https://i.guancha.cn/shiping-banner.jpg']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='毛克疾：美国究竟能在多大程度上容忍印度？', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '毛克疾：美国究竟能在多大程度上容忍印度？', 'publish_time': '2024-06-20 08:26:59', 'author': '郭涵', 'url': 'https://www.guancha.cn/MaoKeJi/2024_06_20_738581.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/19/20240619133335691.jpg', 'https://i.guancha.cn/news/external/2024/06/19/20240619133601871.jpg', 'https://i.guancha.cn/shiping-banner.jpg']}, has

In [38]:
doc_summary_index.ref_doc_info

{'沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”': RefDocInfo(node_ids=['0777402f-2fbc-4357-87ae-17f8c0a6ddb7', 'aa764d29-82f8-45cd-8b53-96c5093c09bf', '49c9cfb9-79b8-4a14-b65c-9f9f0fde3e92', '7b7f553f-7067-4bcf-9cb1-0d4a9f184c30', 'e0a4e135-b842-4631-a1eb-f447df41d0b0'], metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']}),
 '毛克疾：美国究竟能在多大程度上容忍印度？': RefDocInfo(node_ids=['a45e8f6e-db4b-4b86-a103-528c79436834', 'adb5161c-4e91-4cc1-b6bd-3855eb8df573', 'dfa140c0-992f-472b-a06d-331daff77997', '1715363f-2dd0-4982-8da0-3a71dd958780', '50c9fbf1-c8ae-4fb9-bd35-fe0d46137a00', 'b9a1cd87-e1cc-4535-b010-f2eef9bfaf90', 'f5b20010-4332-4464-9f59-8b1e1adf092d', '03a15796-18df-4ecd-ba9d-269f38da77ca'], metadata={'title': '毛克疾：美国究竟能在多大程度上容忍印度？', 'publi

In [39]:
len(doc_summary_index.ref_doc_info)

2

## 增加重复的新闻文档

如果插入重复的文档，将覆盖之前的文档，而不会报错。

In [11]:
%%time

news_data=get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

current doc id: 沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”
CPU times: user 128 ms, sys: 4.03 ms, total: 132 ms
Wall time: 16 s


In [12]:
doc_summary_index.ref_doc_info

{'沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”': RefDocInfo(node_ids=['57734977-662b-4e9d-b621-a01cbade445e', '6bb86945-eb09-480e-a798-8e505504ef51', '619ddb0c-2634-414c-82fa-7fa1d0ad89d7', '50e6ce07-42a6-46ae-a52a-db4d0c188ff1', '1f288a95-3ea8-4aae-baad-5db3858da391', 'a4dcab0f-c8e0-4061-8583-c8456a486081', '57734977-662b-4e9d-b621-a01cbade445e', '6bb86945-eb09-480e-a798-8e505504ef51', '619ddb0c-2634-414c-82fa-7fa1d0ad89d7', '50e6ce07-42a6-46ae-a52a-db4d0c188ff1', '1f288a95-3ea8-4aae-baad-5db3858da391', 'a4dcab0f-c8e0-4061-8583-c8456a486081', 'c5daa467-fd37-49d1-9d2b-70d9fcc03736', '57734977-662b-4e9d-b621-a01cbade445e', '6bb86945-eb09-480e-a798-8e505504ef51', '619ddb0c-2634-414c-82fa-7fa1d0ad89d7', '50e6ce07-42a6-46ae-a52a-db4d0c188ff1', '1f288a95-3ea8-4aae-baad-5db3858da391', 'a4dcab0f-c8e0-4061-8583-c8456a486081', 'c5daa467-fd37-49d1-9d2b-70d9fcc03736', '3205edda-5002-4c49-8546-997c6772f0ac', '57734977-662b-4e9d-b621-a01cbade445e', '6bb86945-eb09-480e-a798-8e505504ef51', '619ddb0c-2634-414c-82

In [13]:
len(doc_summary_index.ref_doc_info)

1